# 🌍 ESG & Financial Intelligence Platform
## AI-Powered Analysis of Corporate Reports (ESG And Annual)

**🎯 What this does:**
- Automatically processes PDF reports from Google Cloud Storage
- Uses Gemini 2.5 Pro to extract financial & ESG metrics
- Generates forecasts using Google's TimesFM 2.0 model
- Analyzes companies: Amgen, Novartis, Target

**📊 Features:**
- ✅ Automated PDF processing from Cloud Storage
- ✅ AI-powered data extraction (Financial + ESG metrics)
- ✅ Revenue forecasting with TimesFM 2.0
- ✅ Multi-company comparative analysis

---
**👆 Click "Runtime" → "Run all" to see the magic happen!**

# 🚀 Setup & Authentication

In [ ]:

print("🌟 Welcome to the ESG & Financial Intelligence Platform!")
print("🔧 Setting up environment...")

# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

# Install required packages
!pip install -q bigframes google-cloud-bigquery plotly seaborn

# Import libraries
import bigframes.pandas as bpd
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

print("✅ Authentication complete!")
print("✅ Libraries installed!")

# Intializing Parameters

In [ ]:
# Replace the below with other company names from this list:
# novartis, amgen, target, bankofmontreal, lloyd, wellsfargo

company1 = "novartis"
company2 ="wellsfargo"
company3 = "lloyd"

PROJECT_ID = "intellitrend-project-dev"   # update if needed
PROJECT_LOCATION="US" # Others are EU, ASIA
CLOUD_RES_CONN = "ghack_conn" #must be all small case


DATASET_ID = "db_reports_insights_annual_esg" #Name of dataset to be created in BQ
OBJ_TABLE_ID = "all_reports_obj_table_metadata" #Objec table name. That holds the metadata of the source files in GCS
RAW_TABLE_ID="all_reports_ai_text_raw"   # the output text that gets extracted based on prmpt - using ML.GENERATE_TEXT

CURATED_TABLE_ID="all_reports_ai_text_curated" # response_text extracted from JSON output
CURATED2_TABLE_ID="all_reports_ai_text_curated2" # the output text that gets extracted based on prmpt - using AI.GENERATE (An alternate way - for comparison)

FINAL_TABLE_ID="all_reports_ai_text_final" # format the output from the curated table
FORECAST_TABLE_ID="all_reports_forecast" # forecast table from the formatted-output

QUALIFIED_CLOUD_RES_CONN = f"{PROJECT_ID}.{PROJECT_LOCATION}.{CLOUD_RES_CONN}"

QUALIFIED_OBJ_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{OBJ_TABLE_ID}"
QUALIFIED_RAW_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE_ID}"

QUALIFIED_CURATED_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{CURATED_TABLE_ID}"
QUALIFIED_CURATED2_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{CURATED2_TABLE_ID}"

QUALIFIED_FINAL_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{FINAL_TABLE_ID}"
QUALIFIED_FORECAST_TABLE_ID= f"{PROJECT_ID}.{DATASET_ID}.{FORECAST_TABLE_ID}"

MODEL_ENDPOINT="gemini-2.5-pro"
MODEL_NAME="gemini_model_25pro"
QUALIFIED_MODEL_ID=f"{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}"

DATA_FILES_PATH="gs://report_insights"

# 🏗️ Project Configuration

In [ ]:

from google.colab import userdata

# Configure BigFrames
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = PROJECT_LOCATION

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"✅ Project configured: {PROJECT_ID}")
print("✅ BigFrames ready for action!")

#  Create Cloud Resource Connection

In [ ]:
!bq mk \
  --connection \
  --project_id={PROJECT_ID} \
  --connection_type=CLOUD_RESOURCE \
  --location={PROJECT_LOCATION} \
  {CLOUD_RES_CONN}

# Fetch Service Account associated with the Cloud Res Connection

In [ ]:
SERVICE_ACCT = !bq show --format=prettyjson \
  --connection \
  --project_id={PROJECT_ID} \
  --location={PROJECT_LOCATION} \
  {CLOUD_RES_CONN} | grep "serviceAccountId" | cut -d '"' -f 4

SERVICE_ACCT_EMAIL = SERVICE_ACCT[0]  # first (and only) line
print(SERVICE_ACCT_EMAIL)

# Give the below roles to the Conn Service Account:
  * objectViewer and  
  * aiplatformUser



In [ ]:
import time

# Use $SERVICE_ACCT_EMAIL so the Python variable expands in the shell
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:$SERVICE_ACCT_EMAIL" \
    --role="roles/storage.objectViewer" \
    --format=none

!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member="serviceAccount:$SERVICE_ACCT_EMAIL" \
    --role="roles/aiplatform.user" \
    --format=none

# Wait ~60 seconds for IAM updates to propagate
time.sleep(60)

# Create the Gemini Model

In [ ]:
# 🤖 Step 0a: Create Gemini 2.5 Pro Model
print("🤖 Creating Gemini 2.5 Pro model...")

create_model_sql = f"""
CREATE OR REPLACE MODEL `{QUALIFIED_MODEL_ID}`
REMOTE WITH CONNECTION `{QUALIFIED_CLOUD_RES_CONN}`
OPTIONS (ENDPOINT = '{MODEL_ENDPOINT}');
"""

try:
    job = client.query(create_model_sql)
    job.result()
    print("🎉 Gemini 2.5 Pro model created successfully!")
except Exception as e:
    print(f"⚠️  Model creation issue: {e}")

# 📁 Create Object Table for PDF Reports

In [29]:


print(f"📁 Creating external table(Object Table): '{OBJ_TABLE_ID}' for PDF reports...")

create_external_table_sql = f"""
CREATE OR REPLACE EXTERNAL TABLE `{QUALIFIED_OBJ_TABLE_ID}`
WITH CONNECTION `{QUALIFIED_CLOUD_RES_CONN}`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['{DATA_FILES_PATH}/*']
);
"""

try:
    job = client.query(create_external_table_sql)
    job.result()
    print("✅ External table created successfully!")

    # Check what files we have
    check_files_sql = f"SELECT uri, size FROM `{QUALIFIED_OBJ_TABLE_ID}`  "
    files_df = bpd.read_gbq(check_files_sql)
    print(f"📄 Found {len(files_df)} files in storage")
    print("Sample files:")
    print(files_df.head())

except Exception as e:
    print(f"⚠️  Note: {e}")
    print("   Make sure your GCS bucket and connection are set up correctly")

📁 Creating external table(Object Table): 'all_reports_obj_table_metadata' for PDF reports...
✅ External table created successfully!
📄 Found 129 files in storage
Sample files:


                                                 uri      size
0  gs://report_insights/amgen/amgen_annualreport_...  12663856
1  gs://report_insights/amgen/amgen_annualreport_...   3651904
2  gs://report_insights/amgen/amgen_annualreport_...   3777808
3  gs://report_insights/amgen/amgen_annualreport_...  12725236
4  gs://report_insights/amgen/amgen_annualreport_...   8154936

[5 rows x 2 columns]


#  🧠 Extract Financial & ESG Data - Using ML.GENERATE_TEXT

In [30]:

print(f"🧠 Processing PDFs with {MODEL_NAME}...")
print(f"This analyzes {company1}, {company2}, and {company3} reports...")

generate_text_sql = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_RAW_TABLE_ID}` AS (
SELECT
  *
FROM
  ML.GENERATE_TEXT(
    MODEL `{QUALIFIED_MODEL_ID}`,
    TABLE `{QUALIFIED_OBJ_TABLE_ID}`,
    STRUCT(
      '''
You are an expert ESG and Financial analyst. Use only the information provided in the document to answer.
        Fetch Financial and Sustainability Details including tabular and image data for each PDF.
        Read the filename. If it is a sustainability report, please get only ESG details. If it is an annual report, get just the financials as below.
        Ensure you fetch details so it has answers for the below:
        Financial Performance
        What is the total revenue this year?
        How has revenue changed compared to last year?
        What is the net profit or loss?
        How have the earnings per share (EPS) changed?
        Are gross margins improving or declining?
        Are operating margins stable or volatile?
        How has net margin shifted year over year?
        Is cash flow from operations positive and consistent?
        How much free cash flow is available?
        How are working capital levels trending?
        Balance Sheet & Liquidity
        What is the total debt level?
        Is the debt-to-equity ratio rising or falling?
        Does the company have enough liquidity to cover short-term liabilities?
        What is the current ratio and quick ratio?
        How much cash and equivalents are available?
        Are interest expenses sustainable?
        How sensitive is the company to interest rate changes?
        What is the level of contingent liabilities?
        Business Strategy
        What are the company’s core growth initiatives?
        Is management investing in new markets or products?
        How much is spent on research and development (R&D)?
        Are acquisitions or divestitures planned?
        Is the company expanding geographically?
        What is the company’s competitive advantage?
        Is the advantage sustainable against competitors?
        Is digital transformation a core part of the strategy?
        Risk Factors
        What macroeconomic risks affect the business?
        What industry-specific risks are highlighted?
        What operational risks are disclosed?
        Are supply chain risks material?
        Are legal or regulatory risks mentioned?
        How is foreign exchange risk managed?
        How dependent is revenue on a few customers?
        Are raw material price risks significant?
        Governance & Leadership
        Is the board majority independent?
        How many women or minorities are on the board?
        Is executive compensation performance-linked?
        Does management own significant equity?
        Are shareholder rights well protected?
        Is succession planning disclosed?
        Has management delivered on past promises?
        Is the auditor independent and credible?
        Shareholder Value
        Is the company paying dividends?
        Is the dividend payout sustainable?
        Is the dividend policy clear?
        Are share buybacks taking place?
        Is return on equity (ROE) improving?
        Is return on assets (ROA) stable?
        Has shareholder equity grown consistently?
        What is the earnings guidance for next year?
        Investor Questions – Sustainability Reports
        Environmental
        What is the total Scope 1 emissions?
        What is the total Scope 2 emissions?
        What is the total Scope 3 emissions?
        Are science-based targets disclosed?
        Has the company committed to net-zero?
        What percentage of energy is renewable?
        Is energy efficiency improving year over year?
        What is the total water withdrawal?
        What is water recycled or reused?
        Is the company exposed to water stress?
        How much total waste is generated?
        How much waste is recycled or diverted from landfill?
        Are hazardous waste levels disclosed?
        How sustainable are raw material sourcing practices?
        Are biodiversity risks addressed?
        Social
        What is the total workforce size?
        What percentage of employees are women?
        What percentage of leadership roles are held by women?
        What is minority representation in the workforce?
        What is the employee turnover rate?
        How many workplace injuries occurred?
        What is the lost-time injury frequency rate?
        Are labor rights respected across the supply chain?
        Is child labor risk disclosed in supply chains?
        Are employee training hours increasing per year?
        Are wages above local living wage levels?
        Is there employee engagement survey data?
        What community development initiatives exist?
        What philanthropic contributions were made?
        Is there disclosure on data privacy and cybersecurity?
        Governance (ESG-specific)
        Is there a dedicated board ESG committee?
        How often does the board review ESG matters?
        Is ESG risk included in enterprise risk management?
        Is executive compensation tied to ESG goals?
        Are ESG targets linked to management incentives?
        Is ESG performance externally assured?
        Are ESG disclosures aligned with GRI or SASB?
        Does the company report under TCFD or ISSB?
        Is CDP reporting publicly available?
        Are stakeholder engagement practices disclosed?
        Risk & Opportunity
        What are the identified physical climate risks?
        What are the identified transition risks?
        Is there disclosure on carbon pricing exposure?
        How dependent is revenue on carbon-intensive activities?
        What sustainability-linked opportunities are identified?
        Are green products or services generating revenue?
        Is access to sustainable finance increasing?
        Are suppliers evaluated for ESG performance?
        Are ESG clauses included in supplier contracts?
        Is there disclosure on product lifecycle impacts?
      ''' AS prompt,

      0 AS temperature,
      8092 AS max_output_tokens
    )
  )
WHERE uri like '%{company1}%' or uri like '%{company2}%' or  uri like '%{company3}%'
);
"""
print("\n\n the SQL \n\n", generate_text_sql)
try:
    print("⏳ Processing... This may take 2-3 minutes")
    job = client.query(generate_text_sql)
    job.result()

    # Check results
    check_sql = f"SELECT COUNT(*) as processed_files FROM `{QUALIFIED_RAW_TABLE_ID}`"
    result = client.query(check_sql).result()
    count = list(result)[0][0]
    print(f"✅ Successfully processed {count} files!")

except Exception as e:
    print(f"⚠️  Processing issue: {e}")

🧠 Processing PDFs with gemini_model_25pro...
This analyzes novartis, wellsfargo, and lloyd reports...


 the SQL 

 
CREATE OR REPLACE TABLE `intellitrend-project-dev.db_reports_insights_annual_esg.all_reports_ai_text_raw` AS (
SELECT
  *
FROM
  ML.GENERATE_TEXT(
    MODEL `intellitrend-project-dev.db_reports_insights_annual_esg.gemini_model_25pro`,
    TABLE `intellitrend-project-dev.db_reports_insights_annual_esg.all_reports_obj_table_metadata`,
    STRUCT(
      '''
You are an expert ESG and Financial analyst. Use only the information provided in the document to answer.
        Fetch Financial and Sustainability Details including tabular and image data for each PDF.
        Read the filename. If it is a sustainability report, please get only ESG details. If it is an annual report, get just the financials as below.
        Ensure you fetch details so it has answers for the below:
        Financial Performance
        What is the total revenue this year?
        How has revenue change

### Alternate Method to ML.GENERATE_TEXT:
# Extract Financial & ESG Data - using AI.GENERATE

In [ ]:

print(f"🧠 Processing PDFs with {MODEL_NAME}...")
print(f"This analyzes {company1}, {company2}, and {company3} reports...")

generate_text_sql2 = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_CURATED2_TABLE_ID}` AS
SELECT
  uri,
  AI.GENERATE(
    CONCAT(
      "You are an expert ESG and Financial analyst. Use only the information provided in the document to answer.\\n",
      "Fetch Financial and Sustainability Details including tabular and image data for each PDF.\\n",
      "Open and Read the PDF: ",
      JSON_VALUE(
        OBJ.GET_ACCESS_URL(
          OBJ.MAKE_REF(uri, '{QUALIFIED_CLOUD_RES_CONN}'),
          'r'
        ),
        '$.access_urls.read_url'
      )
    ),
    connection_id => '{QUALIFIED_CLOUD_RES_CONN}',
    endpoint => '{MODEL_ENDPOINT}',
    output_schema => 'uri STRING, response_text STRING'
  ).response_text AS response_text
FROM `{QUALIFIED_OBJ_TABLE_ID}`
WHERE uri LIKE '%{company1}%' OR uri LIKE '%{company2}%' OR uri LIKE '%{company3}%';
"""

try:
    print("⏳ Processing... This may take 2-3 minutes")
    job = client.query(generate_text_sql2)
    job.result()

    # Check results
    check_sql = f"SELECT COUNT(*) as processed_files FROM `{QUALIFIED_CURATED2_TABLE_ID}`"
    result = client.query(check_sql).result()
    count = list(result)[0][0]
    print(f"✅ Successfully processed {count} files!")

except Exception as e:
    print(f"⚠️  Processing issue: {e}")


# 🧹 Extract Clean Text Response

In [31]:

print("🧹 Cleaning and extracting response text...")

curate_results_sql = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_CURATED_TABLE_ID}` AS
SELECT *,
  ARRAY_TO_STRING(
    ARRAY(
      SELECT JSON_VALUE(part, '$.text')
      FROM UNNEST(JSON_QUERY_ARRAY(
        JSON_QUERY(ml_generate_text_result, '$.candidates[0].content.parts')
      )) AS part
    ),
    ''
  ) AS response_text
FROM
  `{QUALIFIED_RAW_TABLE_ID}`;
"""

try:
    job = client.query(curate_results_sql)
    job.result()
    print("✅ Response text extracted and cleaned!")

    # Show sample of extracted text
    sample_sql = f"""
    SELECT uri, LEFT(response_text, 200) as sample_text
    FROM `{QUALIFIED_CURATED_TABLE_ID}`
    LIMIT 3
    """
    sample_df = bpd.read_gbq(sample_sql)
    print("\n📝 Sample extracted text:")
    for _, row in sample_df.iterrows():
        print(f"File: {row['uri']}")
        print(f"Text: {row['sample_text']}...\n")

except Exception as e:
    print(f"⚠️  Text extraction issue: {e}")

🧹 Cleaning and extracting response text...
✅ Response text extracted and cleaned!

📝 Sample extracted text:
File: gs://report_insights/lloyd/lloyd_annualreport_2021.pdf
Text: ...

File: gs://report_insights/lloyd/lloyd_annualreport_2023.pdf
Text: ...

File: gs://report_insights/lloyd/lloyd_sustainabilityreport_2021.pdf
Text: Based on the "Lloyd's Global Compact 2021 Report," here are the key sustainability details:

### **Environmental**

*   **What is the total Scope 1, 2, and 3 emissions?**
    The report does not provi...



# 📊 Extract Structured Financial & ESG Metrics

In [32]:

print("📊 Generating structured financial and ESG metrics...")

generate_table_sql = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_FINAL_TABLE_ID}` AS
SELECT *,
    DATE(CAST(fiscal_year AS STRING) || "-01-01") AS fiscal_year_date,
    CASE
        WHEN uri LIKE '%{company1}%' THEN '{company1}'
        WHEN uri LIKE '%{company2}%' THEN '{company2}'
        WHEN uri LIKE '%{company3}%' THEN '{company3}'
    END as company_name_formatted
FROM (
    SELECT *
    FROM AI.GENERATE_TABLE(
        MODEL `{QUALIFIED_MODEL_ID}`,
        (
            SELECT CONCAT(
                "You are an expert financial analyst and ESG/Climate analyst. Extract all financial metrics and sustainability metrics and KPIs from the provided text into structured data. ",
                "For Annual reports, focus on getting the financial metrics. And for Sustainability reports focus on getting ESG metrics. ",
                "Normalize numbers by removing commas, spaces, or symbols. ",
                "Only include actual numerical financial data. ",
                "TEXT: ",
                response_text
            ) AS prompt,
            response_text as extracted_text,
            uri AS uri
        FROM `{QUALIFIED_CURATED_TABLE_ID}`
        ),
        STRUCT(
            'uris STRING, report_type STRING, company_name STRING, fiscal_year INT64, revenue_millions FLOAT64, net_income_millions FLOAT64, total_assets_millions FLOAT64, total_liabilities_millions FLOAT64, equity_millions FLOAT64, eps_basic FLOAT64, eps_diluted FLOAT64, dividends_per_share FLOAT64, employee_count INT64, business_segments ARRAY<STRING>, total_ghg_emissions FLOAT64, scope_1_ghg_emissions FLOAT64, scope_2_ghg_emissions FLOAT64, scope_3_ghg_emissions FLOAT64, carbon_intensity FLOAT64, energy_consumption FLOAT64, renewable_energy_usage FLOAT64, water_consumption FLOAT64, waste_generated FLOAT64, waste_recycled FLOAT64, sustainable_financing FLOAT64, reporting_period STRING, page_references ARRAY<STRING>, contexts ARRAY<STRING>, verbatim_quotes ARRAY<STRING>'
            AS output_schema,
            8192 AS max_output_tokens,
            0 AS temperature
        )
    ) t
);
"""

try:
    print("⏳ Structuring data... This may take 3-4 minutes")
    job = client.query(generate_table_sql)
    job.result()
    print("✅ Structured metrics extracted successfully!")

except Exception as e:
    print(f"⚠️  Table generation issue: {e}")

📊 Generating structured financial and ESG metrics...
⏳ Structuring data... This may take 3-4 minutes
✅ Structured metrics extracted successfully!


# 🔮 Revenue Forecasting with TimesFM 2.0

In [33]:

print("🔮 Generating revenue forecasts with TimesFM 2.0...")

forecast_sql = f"""
CREATE OR REPLACE TABLE `{QUALIFIED_FORECAST_TABLE_ID}` AS
SELECT
  *
FROM
  AI.FORECAST(
    TABLE `{QUALIFIED_FINAL_TABLE_ID}`,
    data_col => 'revenue_millions',
    timestamp_col => 'fiscal_year_date',
    model => 'TimesFM 2.0',
    id_cols => ['company_name_formatted'],
    horizon => 5,
    confidence_level => .95
  )
"""

print("⏳ Forecasting...")
job = client.query(forecast_sql)
job.result()
print("✅ Forecast table created successfully!")

🔮 Generating revenue forecasts with TimesFM 2.0...
⏳ Forecasting...
✅ Forecast table created successfully!


In [ ]:
# import requests
# from google.cloud import bigquery

# PROJECT_ID = "intellitrend-project-dev"   # update if needed
# DATASET_ID = "db_reports_insights_annual_esg"
# TABLE_ID = "all_reports_annual_esg"

# # Step 0: Fetch file list from GitHub
# url = "https://api.github.com/repos/intellitrend-global/google_hackathon_bq_ai/contents/annual_esg_reports"
# response = requests.get(url)
# files = response.json()

# pdf_files = [(f["name"], f["download_url"]) for f in files if f["name"].endswith(".pdf")]

# print("📄 Found PDF files on GitHub:")
# for name, link in pdf_files:
#     print(f"   {name} -> {link}")

# # Step 1: Initialize BigQuery client
# client = bigquery.Client(project=PROJECT_ID)

# # Step 2: Ensure dataset exists (create if not)
# dataset_ref = bigquery.DatasetReference(PROJECT_ID, DATASET_ID)
# try:
#     client.get_dataset(dataset_ref)  # Check if it exists
#     print(f"✅ Dataset {DATASET_ID} already exists")
# except Exception:
#     dataset = bigquery.Dataset(dataset_ref)
#     dataset.location = "US"
#     client.create_dataset(dataset, timeout=30)
#     print(f"📦 Created dataset {DATASET_ID}")

# # Step 3: Define schema
# schema = [
#     bigquery.SchemaField("filename", "STRING"),
#     bigquery.SchemaField("uri", "STRING"),
# ]

# # Step 4: Prepare rows
# rows = [{"filename": name, "uri": link} for name, link in pdf_files]

# # Step 5: Load into BigQuery
# job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_TRUNCATE")
# table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

# job = client.load_table_from_json(rows, table_ref, job_config=job_config)
# job.result()

# print(f"✅ Table {table_ref} created/updated with {len(pdf_files)} GitHub files")
